In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
103,2024-08-30,Eua Wnba,20:30,Chicago Sky F,Indiana Fever F,2.71,1.47,165.5,1.88,1.90,4.5,1.94,1.82,jXZrPftn,0.369004,0.680272,0.531915,0.526316,0.049276,0.6,0.6,NaN,NaN,214.348,132.016345,0.615897,177.152,32.320790,0.182447,225.712,158.018,136.50,145.32,0.0,0.0,0.0,0.0,0.419527,0.007483,0.045134,-3.38,-0.676,-2.529586,0.0,0.0,0.0,2.65,0.53,0.886792,0.0,0.0,0.0
104,2024-08-30,Eua Wnba,20:30,Dallas Wings F,Minnesota Lynx F,3.38,1.33,169.5,1.86,1.92,6.5,1.97,1.81,YePmOEeh,0.295858,0.751880,0.537634,0.520833,0.047738,0.0,0.0,NaN,NaN,168.876,59.413114,0.351815,165.870,51.994033,0.313463,0.000,0.000,183.06,138.84,0.0,0.0,0.0,0.0,0.615528,0.022448,0.059861,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
105,2024-08-30,Eua Wnba,23:00,Seattle Storm F,New York Liberty F,2.12,1.73,162.5,1.81,1.95,1.5,1.94,1.82,tMSeMhQ4,0.471698,0.578035,0.552486,0.512821,0.049733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,137.41,189.90,0.0,0.0,0.0,0.0,0.143258,0.052657,0.045134,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
106,2024-08-30,Filipinas Copa Dos Governadores,06:00,Rain Or Shine Paint.,Phoenix Fuelmasters,1.05,8.90,214.5,1.87,1.87,-16.5,1.99,1.73,pv1M1ANj,0.952381,0.112360,0.534759,0.534759,0.064741,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,169.36,0.00,0.0,0.0,0.0,0.0,1.115736,0.000000,0.098843,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
107,2024-08-30,Filipinas Copa Dos Governadores,08:30,Ginebra Kings,Blackwater Bossing,1.19,4.44,199.5,1.81,1.93,-10.5,1.94,1.77,bJr9PDFc,0.840336,0.225225,0.552486,0.518135,0.065561,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.816376,0.045376,0.064802,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
108,2024-08-30,Mundo Amistoso Interclubes,12:00,DEAC (Hun),Spisski Rytieri (Svk),1.37,2.90,158.5,1.83,1.83,-6.5,1.83,1.83,fwMDShGO,0.729927,0.344828,0.546448,0.546448,0.074755,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.506732,0.000000,0.000000,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
109,2024-08-30,Mundo Amistoso Interclubes,13:00,Decin (Cze),Bayreuth (Ger),1.50,2.34,163.5,1.82,1.84,-5.5,1.98,1.70,prJvshVH,0.666667,0.427350,0.549451,0.543478,0.094017,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.309359,0.007728,0.107603,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
110,2024-08-30,Mundo Amistoso Interclubes,13:30,Chartres (Fra),Orleans (Fra),3.55,1.23,150.5,1.83,1.83,7.5,1.98,1.70,dIjoP2Fk,0.281690,0.813008,0.546448,0.546448,0.094698,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.686397,0.000000,0.107603,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
111,2024-08-30,Mundo Amistoso Interclubes,14:00,Prievidza (Svk),Brno (Cze),1.38,2.70,161.5,1.84,1.82,-6.5,1.94,1.74,OSUVrE05,0.724638,0.370370,0.543478,0.549451,0.095008,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.457540,0.007728,0.076859,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0
112,2024-08-30,Mundo Amistoso Interclubes,14:00,Szolnoki Olaj (Hun),CSM Oradea (Rou),2.50,1.51,158.5,1.83,1.83,4.5,1.83,1.83,bBS4UEoC,0.400000,0.662252,0.546448,0.546448,0.062252,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.349145,0.000000,0.000000,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.00,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
